## Creating schools.csv

1. Install packages
2. Create cities.csv with full state name/ city column to use in getting school information
3. For persisitance creating a schools csv using selenium to get school information from greatschools.org
4. Clean csv for use in schools endpoint

### 1. Import necessary libraries

In [ ]:
import os
import requests
import csv
import json
import pandas as pd
from state_abbr import us_state_abbrev as abbr
from selenium import webdriver
import urllib.parse

### 2.  Create cities.csv with full state name/ city column to use in getting school information

In [ ]:
pwd = os.getcwd()

# create city state list
cities = pd.read_excel('notebooks/datasets/data/schools/csv/List of Cities.xlsx')

# just get the second and third colun
cities = cities[['Unnamed: 1','Unnamed: 2']]

# create new dictionary with reversed key, value pairs
full = dict(map(reversed, abbr.items()))

# map state abbreviations to full name
cities['states'] = cities['Unnamed: 2'].map(full)

# making sure state/city combo conform to url format of "-" for " "
cities['states'] = cities['states'].str.strip()
cities['states'] = cities['states'].str.replace(" ", "-")
cities['Unnamed: 1'] = cities['Unnamed: 1'].str.replace(" ", "-")

# remove extraneous header rows
cities = cities.iloc[2:]
cities['city'] = (cities['states'] + '/'+ cities['Unnamed: 1']).str.lower()
print(cities.head())

# persist by creating new csv
cities.to_csv('notebooks/datasets/data/schools/csv/cities.csv')

### 3. For persisitance creating a schools csv using selenium to get school information from greatschools.org

In [ ]:
# using selenium to get school information
driver = webdriver.Chrome()

In [ ]:
# url for greatschools pre_url and post_url (with state/city inbetween)
url_pre = 'http://www.greatschools.org/'
url_post = '/schools/?tableView=Overview&view=table'

# Call cities csv to get cities stored in database
cities = pd.read_csv('notebooks/datasets/data/schools/csv/cities.csv')

In [ ]:
# Looping through each city in the file

# create empty dataframe
df = pd.DataFrame()

for i in cities['city']:
    endpoint = url_pre + urllib.parse.quote(i) + url_post
    print("Fetching ", endpoint)
    driver.get(endpoint)
    html = driver.page_source
    table = pd.read_html(html)
    # appending to dataframe all the schhol information for current cities
    df = df.append(table[0])

driver.close()

In [ ]:
# 5. For persisitance creating a schools csv
df.to_csv('notebooks/datasets/data/schools/csv/schools.csv')

### 4. Clean csv for use in schools endpoint

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

/Users/jisha/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('csv/schools.csv')

In [3]:
# Drop unused columns
df = df.drop(columns=['Unnamed: 0', 'Reviews'])

In [4]:
# Extract numerical rating
df[['Rating', 'School']] = df['School'].str.split('/10', n=2, expand=True)

In [5]:
# Extract score 
# https://stackoverflow.com/questions/4416425/how-to-split-string-with-some-separator-but-without-removing-that-separator-in-j
df['Score'] = df['School'].str.split('(?<=[a-z])(?=[A-Z])')
df['Score'] = df['Score'].str[0]

In [6]:
# Cleaning strings

# School
df['School'] = df['School'].str.replace('Top rated', '')
df['School'] = df['School'].str.replace('Above average', '')
df['School'] = df['School'].str.replace('Average', '')
df['School'] = df['School'].str.replace('Below average', '')
df['School'] = df['School'].str.replace('None', '')
df['School'] = df['School'].str.replace(' Homes for sale', '')
df['School'] = df['School'].str.replace('2 awardsAwards & BadgesCollege Success Award2019, 2020See all winners in ', '')
df['School'] = df['School'].str.replace('3 awardsAwards & BadgesCollege Success Award2019, 2020See all winners in ', '')

# Score
df['Score'] = df['Score'].str.replace('Below average7th & 8th Grade Center402 North S Street, Muskogee, OK, 74403 Homes for sale', 'Below average')
df['Score'] = df['Score'].str.replace('Below average7th & 8th Grade Center402 North S Street, Muskogee, OK, 74403 Homes for sale', 'Below average')
df['Score'] = df['Score'].str.replace('Average12th Street Elementary6501 South 12th Street, Portage, MI, 49024 Homes for sale', 'Average')
df['Score'] = df['Score'].str.replace('Below average21st Century Preparatory School1220 Mound Avenue, Racine, WI, 53404 Homes for sale', 'Below average')
df['Score'] = df['Score'].str.replace('Below averagee', 'Below average')
df['Score'] = df['Score'].str.replace('Averagedu', 'Average')


In [7]:
df['Address'] = df['School'].str.split('(?<=[a-z])(?=[0-9])')

In [8]:
# School Name 
df['School'] = df['Address'].str[0]

In [9]:
# Finding length because there are anomalies with the information in the address column
df['Length'] = df['Address'].apply(lambda x: len(x) if x != None else 0 )
# 2 is the expected length -> school and address
df['Length'].unique()

array([4, 2, 0, 1, 3])

In [10]:
# These are the rows that need extra help
unrated = df.loc[df['Length'] == 0]
miscellaneous = df.loc[df['Length'] == 3]

In [11]:
# This where the address should be in most cases except the ones above
df['Address'] = df['Address'].str[-1]